In [ ]:
#@markdown Setup dependencies (Colab only)
%%capture
try:
    import papyrus_scripts
except:
    !pip uninstall papyrus-scripts -y
    !pip install rdkit-pypi
    !pip install https://github.com/OlivierBeq/Papyrus-scripts/tarball/master --no-cache-dir
    get_ipython().kernel.do_shutdown(True)

# Simple examples: Using Papyrus scripts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierBeq/Papyrus-scripts/blob/master/notebook_examples/simple_examples.ipynb)

Herein it is assumed that the Papyrus <a href="https://doi.org/10.4121/16896406">bioactivity data</a> hosted on 4TU or Google Drive was ***NOT*** dowloaded.

In [ ]:
%%html
<style>
table {align:left;display:block}
</style>

## Download the data


One can easily download (part of) the Papyrus data using *papyrus* command (*download* subcommand).<br/>
```
papyrus download --version latest -s without -S -d mold2 -d unirep
```



It can also be carried out programatically with the following.

In [ ]:
from papyrus_scripts.download import download_papyrus

The default behaviour is to:
- download the curated 2D data (*nostereo* argument)
- omit the not curated 3D data (*stereo*)
- omit molecular structures (*structures*)
- download all molecular/sequence descriptors (*descriptors*)

Let's download the 2D curated data with molecular structures and only descriptors used below (total of 2.80 GB).

In [ ]:
download_papyrus(version='latest', structures=True, descriptors=['mold2', 'unirep'])

Latest version: 05.5
Number of files to be donwloaded: 9
Total size: 2.80GB


Donwloading version 05.5:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

At the time of writing (Apr. 26th, 2022) the latest version available is 05.5.

A custom directory to donwload the data to can be indicated with the *outdir* argument.

## Reading Papyrus files

Functions can be found under *papyrus_scripts.reader* to facilitate the dataset being read from disk.

In [ ]:
from papyrus_scripts.reader import read_papyrus, read_protein_set

### Bioactivity data

Let's first read the bioactivity data.

We will use the *read_papyrus* function to read the bioactivity data as a pandas dataframe. <br/>
Let us first demonstrate the use of the function on systems with limited RAM (less than 50GB).

We first ensure to read the standardized data without stereochemistry in chunks of ten thousand lines.<br/>
Additionally we ensure the *source_path* is that given to the *download_papyrus* function above through the *outdir* argument.

In [ ]:
sample_data = read_papyrus(is3d=False, chunksize=10000, source_path=None)

The return value is an iterator of Pandas dataframes of maximum ten thousand rows each.<br/>
Let's extract the first chunk as a pandas dataframe and have a look at few rows.

In [ ]:
chunk1 = next(sample_data)
chunk1.head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,type_other,Activity_class,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD
0,AAAAEENPAALFRN_on_P49654_WT,High,ChEMBL30,CHEMBL492934,COc1cc(C(C)C)c(Oc2cnc(NCCS(C)(=O)=O)nc2N)cc1I,AAAAEENPAALFRN,AAAAEENPAALFRN-UHFFFAOYSA-N,InChI=1S/C17H23IN4O4S/c1-10(2)11-7-14(25-3)12(...,"""AuxInfo=1/1/N:7,8,1,19,16,17,4,25,12,6,5,26,9...",P49654_WT,...,0,NaN,=,7.800,7.800,0.000,0.000,1.0,7.800,0.000
1,AAAAEENPAALFRN_on_P56373_WT,Low,ChEMBL30,CHEMBL492934,COc1cc(C(C)C)c(Oc2cnc(NCCS(C)(=O)=O)nc2N)cc1I,AAAAEENPAALFRN,AAAAEENPAALFRN-UHFFFAOYSA-N,InChI=1S/C17H23IN4O4S/c1-10(2)11-7-14(25-3)12(...,"""AuxInfo=1/1/N:7,8,1,19,16,17,4,25,12,6,5,26,9...",P56373_WT,...,0,NaN,=,7.400,7.400,0.000,0.000,1.0,7.400,0.000
2,AAAAEENPAALFRN_on_Q9UBL9_WT,Low,ChEMBL30,CHEMBL492934,COc1cc(C(C)C)c(Oc2cnc(NCCS(C)(=O)=O)nc2N)cc1I,AAAAEENPAALFRN,AAAAEENPAALFRN-UHFFFAOYSA-N,InChI=1S/C17H23IN4O4S/c1-10(2)11-7-14(25-3)12(...,"""AuxInfo=1/1/N:7,8,1,19,16,17,4,25,12,6,5,26,9...",Q9UBL9_WT,...,0,NaN,=,7.400,7.400,0.000,0.000,1.0,7.400,0.000
3,AAAAKTROWFNLEP_on_P49137_WT,Low,ChEMBL30,CHEMBL246893,CC1CNC(=O)c2c1c1c(ccc(C(=O)N(C)C)c1)[nH]2,AAAAKTROWFNLEP,AAAAKTROWFNLEP-UHFFFAOYSA-N,InChI=1S/C15H17N3O2/c1-8-7-16-14(19)13-12(8)10...,"""AuxInfo=1/1/N:1,17,18,12,11,19,3,2,13,9,10,8,...",P49137_WT,...,0,NaN,<,4.699,4.699,0.000,0.000,1.0,4.699,0.000
4,AAAAZQPHATYWOK_on_P00533_WT,High,Sharma2016;Sharma2016;ChEMBL30,4277046;4277046;CHEMBL175513,CCOc1c(NC(=O)C=CCN(C)C)cc2c(Nc3cc(Cl)c(OCc4nc5...,AAAAZQPHATYWOK,AAAAZQPHATYWOK-UHFFFAOYSA-N,InChI=1S/C32H29ClN6O3S/c1-4-41-28-16-25-22(15-...,"""AuxInfo=1/1/N:1,13,14,2,32,31,10,33,30,9,36,3...",P00533_WT,...,0,NaN,=,6.063; 6.726; 6.730,6.506,0.313,0.222,3.0,6.726,0.004


If you are sure your hardware can handle loading all the data, then you can drop *chunksize*.<br/>
Then the return value is a pandas dataframe.

Below, we will show how to use:
  - a pandas dataframe (by calling our methods on *chunk1*)
  - an iterator of pandas dataframes (by calling our methods on *sample_data*)

### Protein target data

But for now let's focus on protein data:<br/>
Information about the protein targets is available from a different file and can be loaded as easily as was demonstrated above.<br/>
This file being very limited in size, chunking is not needed. 

In [ ]:
protein_data = read_protein_set(source_path=None)
protein_data.head()

,target_id,HGNC_symbol,UniProtID,Status,Organism,Classification,Length,Sequence
0,P47747_WT,,HRH2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,359,MAFNGTVPSFCMDFTVYKVTISVILIILILVTVAGNVVVCLAVGLN...
1,B0FL73_WT,,B0FL73_CAVPO,unreviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,467,MGAGVLALGASEPCNLSSTAPLPDGAATAARLLVPASPPASLLPPT...
2,Q8K4Z4_WT,,ADRB2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,418,MGHLGNGSDFLLAPNASHAPDHNVTRERDEAWVVGMAIVMSLIVLA...
3,P97266_WT,,OPRM_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,98,YTKMKTATNIYIFNLALADALATSTLPFQSVNYLMGTWPFGTILCK...
4,P41144_WT,,OPRK_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,380,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...


## Filtering Papyrus

The data contained in the dataset can be filtered very easily using functions under *papyrus_scripts.preprocess*.<br/>
All filtering functions start with ***keep_***.

In [ ]:
from papyrus_scripts.preprocess import (keep_quality, keep_source, keep_type,
                                        keep_organism, keep_accession, keep_protein_class,
                                        keep_match, keep_contains
                                       )

**The strength of the Papyrus scripts is that the data can be filtered whether chunked or not.** The only difference:
  - when using chunked data, call *consume_chunks* once all filters are applied to reconstiture a pandas dataframe

In addition to the 8 **keep_** functions above, one can filter compounds similar to a reference using the *keep_similar* and *keep_substructures* functions (see [advanced_querying.ipynb](https://github.com/OlivierBeq/Papyrus-scripts/blob/master/notebook_examples/advanced_querying.ipynb)).

### Filtering pandas dataframes

Let's first keep the data with quality 'medium' and above (namely 'high' and 'medium').

In [ ]:
filter1 = keep_quality(data=chunk1, min_quality='medium')

<u>Using <a href="https://www.ebi.ac.uk/chembl/visualise/">ChEMBL's protein target tree</a> is encouraged for this part.</u><br/>
<br/>
We will then filter out any protein not belonging to these two classes:
* Ligand-gated ion channels
* SLC superfamily of solute carriers

For this filter, passing protein information is required (the same applies for *keep_organism* and *keep_accession*).

In [ ]:
filter2 = keep_protein_class(data=filter1, protein_data=protein_data, classes=[{'l2': 'Ligand-gated ion channels'}, {'l3': 'SLC superfamily of solute carriers'}])

We now keep only K<sub>i</sub> and K<sub>D</sub> data.<br/>
Here we will pass filter1 to the next *keep_* funtion.

In [ ]:
filter3 = keep_type(data=filter2, activity_types=['Ki', 'KD'], njobs=1)

We finally keep only human and rat data (protein information is also required here).

In [ ]:
filter4 = keep_organism(data=filter3, protein_data=protein_data, organism=['Human', 'Rat'], generic_regex=True)

Let us have a look at the filtered data.

In [ ]:
filter4.head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,Classification,Organism
0,AAEKULYONKUBOZ_on_P23975_WT,Medium,ChEMBL30,CHEMBL14144,CN1C2CCC1C(C(=O)Oc1ccccc1)C(c1ccc(Cl)cc1)C2,AAEKULYONKUBOZ,AAEKULYONKUBOZ-UHFFFAOYSA-N,InChI=1S/C21H22ClNO2/c1-23-16-11-12-19(23)20(2...,"""AuxInfo=1/0/N:1,14,13,15,12,16,19,24,20,23,4,...",P23975_WT,...,=,6.620,6.62,0.0,0.0,1.0,6.62,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
1,ABAOWFCFAADMKA_on_P23975_WT,Medium,ChEMBL30,CHEMBL456654,c1cnc(CC2(c3ccc4[nH]ccc4c3)CCNC2)cc1,ABAOWFCFAADMKA,ABAOWFCFAADMKA-UHFFFAOYSA-N,InChI=1S/C18H19N3/c1-2-8-20-16(3-1)12-18(7-10-...,"""AuxInfo=1/0/N:21,1,20,8,9,13,16,2,12,17,15,5,...",P23975_WT,...,=,8.100,8.10,0.0,0.0,1.0,8.10,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
2,ABARJWHHNCOEQT_on_P23975_WT,High,ChEMBL30,CHEMBL141741,CNCCC(Oc1cccc2ccccc12)c1c(C)cccc1,ABARJWHHNCOEQT,ABARJWHHNCOEQT-UHFFFAOYSA-N,InChI=1S/C21H23NO/c1-16-8-3-5-11-18(16)21(14-1...,"""AuxInfo=1/0/N:19,1,21,13,22,14,9,20,12,10,23,...",P23975_WT,...,=,6.960,6.96,0.0,0.0,1.0,6.96,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
3,ABXUYGWOEIXKBD_on_P23975_WT,Medium,ChEMBL30,CHEMBL275212,COC(=O)C1C(c2ccc(C=CI)cc2)CC2CCC1N2C,ABXUYGWOEIXKBD,ABXUYGWOEIXKBD-UHFFFAOYSA-N,InChI=1S/C18H22INO2/c1-20-14-7-8-16(20)17(18(2...,"""AuxInfo=1/0/N:22,1,9,14,8,15,18,19,11,12,16,1...",P23975_WT,...,=,6.350,6.35,0.0,0.0,1.0,6.35,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)
4,AAADQEFQWNJHOZ_on_P23975_WT,High,ChEMBL30,CHEMBL336100,OC1(c2ccc(Cl)cc2)c2c(C3=NCCN31)c(F)ccc2,AAADQEFQWNJHOZ,AAADQEFQWNJHOZ-UHFFFAOYSA-N,InChI=1S/C16H12ClFN2O/c17-11-6-4-10(5-7-11)16(...,"""AuxInfo=1/0/N:20,21,19,4,9,5,8,14,15,3,6,10,1...",P23975_WT,...,=,9.05,9.05,0.0,0.0,1.0,9.05,0.0,Transporter->Electrochemical transporter->SLC ...,Homo sapiens (Human)


In [ ]:
print(f'Number of activity points: {filter4.shape[0]}')

Number of activity points: 52


Remember that this result comes from only the first chunk of the entire dataset.

One can now save this dataframe like any other pandas object.

### Filtering iterators of dataframes

Now that the filtering capacity of the Papyrus scripts have been demonstrated for entire dataframes, we can try with chunked iterators.

Let's first reinstanciate sample data. This time we will use a chunk size of 1,000,000.

In [ ]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)

For this will will go through the same filters as above but iterate over the entire dataset.

In [ ]:
filter1_it = keep_quality(data=sample_data, min_quality='medium')
filter2_it = keep_protein_class(data=filter1_it, protein_data=protein_data, classes=[{'l2': 'Ligand-gated ion channels'}, {'l3': 'SLC superfamily of solute carriers'}])
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])
filter4_it = keep_organism(data=filter3_it, protein_data=protein_data, organism=['Human', 'Rat'], generic_regex=True)

The filters do not get applied directly on chunked iterators and one can easily check that *filter4_it* is not a pandas dataframe.

In [ ]:
filter4_it

<generator object _chunked_keep_organism at 0x7f700b46ead0>

To apply the filters on the entire iterator, one needs to call *consume_chunks*.<br/>
This function can be found under *papyrus_scripts.preprocess* just like the *keep_* functions used for filtering.

In [ ]:
from papyrus_scripts.preprocess import consume_chunks

In order to follow progress of the filtering process, one needs to pass the total number of chunks the filters will go through.<br/>
$Total = \displaystyle \Bigl \lceil\frac{Size_{dataset}}{chunksize}\Bigl \rceil $<br/>

In version 05.5 of the Papyrus dataset the number of compound-protein activity points depends on whether stereochemistry is used or not **(remember we discourage its usage)**.<br/>


| Stereochemistry | Size of dataset | 
| :--- | :---: |
| Without | 59,775,912 |
| With (strongly discouraged) | 61,097,228 |

In this example $Total = \displaystyle \Bigl \lceil \frac{59,775,912}{1,000,000}\Bigl \rceil = 60 $<br/>

In [ ]:
filtered_data = consume_chunks(filter4_it, progress=True, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

Although this may take up to 30 minutes to filter the entire dataset, this is the ideal way to work with this dataset on laptops.

In [ ]:
print(f'Number of activity points: {filtered_data.shape[0]}')

Number of activity points: 10614


We hope these simple examples demonstrated how the Papyrus data can easily be filtered.
Let's now focus on the modelling

## Modelling the bioactivity data

The Papyrus scripts allow for both quantitative structure-activity relationship (QSAR) and proteochemometrics (PCM) modelling.<br/>
All functions related to modelling can be found under *papyrus_scripts.modelling*.

**Disclaimer:**<br/>
For now, only precomputed molecular descriptors can be used, preventing the use of models outside of Papyrus.<br/>
This major flaw will be soon fixed.

### QSAR models

In [ ]:
from papyrus_scripts.modelling import qsar
import xgboost

Let us first restrict the data that we just extracted from Papyrus to the human serotonin receptor (accession P31645).

In [ ]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)
filter1_it = keep_accession(sample_data, 'P31645')
filter2_it = keep_quality(data=filter1_it, min_quality='medium')
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])

In [ ]:
SLC6A4_data = consume_chunks(filter3_it, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

We will first create a regression model predicting the average pActivity values of a compound-target pair (i.e. *pchembl_value_Mean*).<br/>
Let's ensure the *descriptor_path* below is that given to the *outdir* argument of the *download_papyrus* function above.

In [ ]:
reg_model = xgboost.XGBRegressor(verbosity=0)

In [ ]:
reg_results, trained_reg_model = qsar(data=SLC6A4_data,
                                      endpoint='pchembl_value_Mean',
                                      num_points=30,
                                      delta_activity=2,
                                      descriptors='mold2',
                                      descriptor_path=None,
                                      descriptor_chunksize=50000,
                                      activity_threshold=6.5,
                                      model=reg_model,
                                      folds=5,
                                      stratify=False,
                                      split_by='Year',
                                      split_year=2013,
                                      test_set_size=0.30,
                                      cluster_method=None,
                                      custom_groups=None,
                                      random_state=1234,
                                      verbose=True)

Loading molecular descriptors: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
reg_results

number        R2       MSE      RMSE      MSLE  \
target                                                                   
P31645_WT Fold 1    340.000000  0.580407  0.652239  0.807613  0.010462   
          Fold 2    340.000000  0.557454  0.677939  0.823370  0.010450   
          Fold 3    339.000000  0.504417  0.694161  0.833163  0.010431   
          Fold 4    339.000000  0.513831  0.703061  0.838487  0.010790   
          Fold 5    339.000000  0.581554  0.686914  0.828803  0.011011   
          Mean      339.400000  0.547533  0.682863  0.826287  0.010629   
          SD          0.447214  0.029809  0.015885  0.009658  0.000213   
          Test set  312.000000  0.155997  1.031592  1.015673  0.015760   

                       RMSLE       MAE  Explained Variance  Max Error  \
target                                                                  
P31645_WT Fold 1    0.102284  0.634414            0.583283   2.657424   
          Fold 2    0.102226  0.639356            0.557867   2.608378   
          Fold 3    0.102132  0.644731            0.505198   3.762562   
          Fold 4    0.103876  0.650108            0.519101   2.484164   
          Fold 5    0.104934  0.650517            0.588812   2.680803   
          Mean      0.103090  0.643825            0.550852   2.838666   
          SD        0.001027  0.005681            0.030644   0.426235   
          Test set  0.125540  0.828603            0.171819   3.090501   

                    Mean Poisson Distrib  Mean Gamma Distrib  Pearson r  \
target                                                                    
P31645_WT Fold 1                0.092378            0.013333   0.772771   
          Fold 2                0.094541            0.013437   0.755860   
          Fold 3                0.096742            0.013692   0.711255   
          Fold 4                0.098324            0.013981   0.723922   
          Fold 5                0.097488            0.014097   0.782011   
          Mean                  0.095895            0.013708   0.749164   
          SD                    0.001973            0.000271   0.025017   
          Test set              0.144501            0.020467   0.415540   

                    Spearman r  Kendall tau  R2_0 (pred. vs. obs.)  \
target                                                               
P31645_WT Fold 1      0.773205     0.583661               0.582030   
          Fold 2      0.751179     0.565002               0.558512   
          Fold 3      0.709075     0.528609               0.505358   
          Fold 4      0.726824     0.541035               0.520181   
          Fold 5      0.778285     0.587943               0.586172   
          Mean        0.747714     0.561250               0.550450   
          SD          0.024241     0.021225               0.029687   
          Test set    0.377786     0.259750               0.171248   

                    R'2_0 (obs. vs. pred.)  k slope (pred. vs obs.)  \
target                                                                
P31645_WT Fold 1                  0.023873                 0.994862   
          Fold 2                 -0.054101                 0.982790   
          Fold 3                 -0.030465                 0.982535   
          Fold 4                 -0.081575                 0.975088   
          Fold 5                 -0.050044                 0.999018   
          Mean                   -0.038462                 0.986858   
          SD                      0.032112                 0.008018   
          Test set               -2.959974                 0.963168   

                    k' slope (obs. vs pred.)  
target                                        
P31645_WT Fold 1                    0.993210  
          Fold 2                    1.005400  
          Fold 3                    1.004953  
          Fold 4                    1.012980  
          Fold 5                    0.988066  
          Mean                      1.000922  
          SD 

When looking at average R<sup>2</sup>, performance over cross-validation is correct but the model show very little capacity to predict the temporally split test set.

To train a classifier, all that is needed is to change the type of model.

In [ ]:
cls_model = xgboost.XGBClassifier(verbosity=0)

In [ ]:
cls_results, trained_cls_model = qsar(data=SLC6A4_data,
                                      endpoint='pchembl_value_Mean',
                                      num_points=30,
                                      delta_activity=2,
                                      descriptors='mold2',
                                      descriptor_path=None,
                                      descriptor_chunksize=50000,
                                      activity_threshold=6.5,
                                      model=cls_model,
                                      folds=5,
                                      stratify=False,
                                      split_by='Year',
                                      split_year=2013,
                                      test_set_size=0.30,
                                      cluster_method=None,
                                      custom_groups=None,
                                      random_state=1234,
                                      verbose=True)

Loading molecular descriptors: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
cls_results

MCC      0:1       ACC      BACC  Sensitivity  \
target                                                                   
P31645_WT Fold 1    0.534390  240:100  0.817647  0.739583     0.550000   
          Fold 2    0.475926   249:91  0.808824  0.712587     0.505495   
          Fold 3    0.492580   245:94  0.811209  0.715306     0.500000   
          Fold 4    0.484068   240:99  0.802360  0.703157     0.464646   
          Fold 5    0.595201   242:97  0.843658  0.757689     0.556701   
          Mean      0.516433        -  0.816740  0.725664     0.515368   
          SD        0.040382        -  0.013073  0.018277     0.031129   
          Test set  0.215737   236:76  0.756410  0.580285     0.236842   

                    Specificity       PPV       NPV        F1     AUC 0  \
target                                                                    
P31645_WT Fold 1       0.929167  0.763889  0.832090  0.639535  0.148417   
          Fold 2       0.919679  0.696970  0.835766  0.585987  0.132001   
          Fold 3       0.930612  0.734375  0.829091  0.594937  0.151585   
          Fold 4       0.941667  0.766667  0.810036  0.578616  0.181229   
          Fold 5       0.958678  0.843750  0.843636  0.670807  0.086692   
          Mean         0.935960  0.761130  0.830124  0.613977  0.139985   
          SD           0.012169  0.044153  0.010189  0.032350  0.028316   
          Test set     0.923729  0.500000  0.789855  0.321429  0.405274   

                       AUC 1  
target                        
P31645_WT Fold 1    0.851583  
          Fold 2    0.867999  
          Fold 3    0.848415  
          Fold 4    0.818771  
          Fold 5    0.913308  
          Mean      0.860015  
          SD        0.028316  
          Test set  0.594726

Looking at the active to inactive ratio (i.e. A:N) one can clearly identify the reason of this low prediction performance over the test set.<br/>
Oversampling and/or undersampling techniques could help the model better identify the boundary between actives and inactives in the mmecular descriptor space.<br/>
However the use of such techniques is not the focus here.

### PCM models

In [ ]:
from papyrus_scripts.modelling import pcm

Let us see if including Rat data improves the quality of the model.

In [ ]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)
filter1_it = keep_accession(sample_data, ['P31645', 'P31652'])
filter2_it = keep_quality(data=filter1_it, min_quality='medium')
filter3_it = keep_type(data=filter2_it, activity_types=['Ki', 'KD'])

In [ ]:
SLC6A4_human_rat = consume_chunks(filter3_it, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

Let's ensure the *descriptor_path* below is that given to the *outdir* argument of the *download_papyrus* function above.

In [ ]:
pcm_reg_model = xgboost.XGBRegressor(verbosity=0)

In [ ]:
pcm_reg_results, pcm_reg_trained_model = pcm(data=SLC6A4_human_rat,
                                             endpoint='pchembl_value_Mean',
                                             num_points=30,
                                             delta_activity=2,
                                             mol_descriptors='mold2',
                                             mol_descriptor_path=None,
                                             mol_descriptor_chunksize=50000,
                                             prot_sequences_path=None,
                                             prot_descriptors='unirep',
                                             prot_descriptor_path=None,
                                             prot_descriptor_chunksize=50000,
                                             activity_threshold=6.5,
                                             model=pcm_reg_model,
                                             folds=5,
                                             stratify=False,
                                             split_by='Year',
                                             split_year=2013,
                                             test_set_size=0.30,
                                             cluster_method=None,
                                             custom_groups=None,
                                             random_state=1234,
                                             verbose=True)

Loading molecular descriptors: 0it [00:00, ?it/s]

Loading protein descriptors: 0it [00:00, ?it/s]

Fitting model:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pcm_reg_results

,number,R2,MSE,RMSE,MSLE,RMSLE,MAE,Explained Variance,Max Error,Mean Poisson Distrib,Mean Gamma Distrib,Pearson r,Spearman r,Kendall tau,R2_0 (pred. vs. obs.),R'2_0 (obs. vs. pred.),k slope (pred. vs obs.),k' slope (obs. vs pred.)
Fold 1,753.000000,0.606211,0.605585,0.778193,0.009173,0.095775,0.623949,0.606288,2.775585,0.083799,0.011827,0.795078,0.799634,0.599757,0.606884,0.041993,0.984664,1.004386
Fold 2,753.000000,0.590141,0.560038,0.748357,0.008257,0.090869,0.582737,0.590141,2.891541,0.076390,0.010609,0.776346,0.769685,0.577590,0.590281,0.084164,0.988024,1.001869
Fold 3,753.000000,0.532261,0.718617,0.847713,0.011385,0.106703,0.652612,0.533351,3.907982,0.101642,0.014687,0.735989,0.750343,0.554585,0.532784,-0.117698,0.990551,0.996132
Fold 4,752.000000,0.573919,0.631884,0.794911,0.009590,0.097926,0.617346,0.575788,2.608555,0.087533,0.012340,0.764234,0.760647,0.568646,0.576741,0.093034,0.980021,1.008820
Fold 5,752.000000,0.566278,0.678797,0.823892,0.010528,0.102604,0.641529,0.566558,3.013564,0.094844,0.013521,0.763478,0.766329,0.573221,0.566286,-0.061014,0.987975,0.999529
Mean,752.600000,0.573762,0.638984,0.798613,0.009787,0.098775,0.623635,0.574425,3.039445,0.088842,0.012597,0.767025,0.769328,0.574760,0.574595,0.008096,0.986247,1.002147
SD,0.447214,0.022729,0.050539,0.031639,0.000988,0.004996,0.021879,0.022409,0.414767,0.007980,0.001280,0.017595,0.015074,0.013414,0.022768,0.076103,0.003315,0.003930
Test set,632.000000,0.305520,0.950761,0.975070,0.016073,0.126781,0.787663,0.318992,3.397805,0.139213,0.020729,0.564794,0.545691,0.384183,0.318892,-1.170678,1.000164,0.980907


As with QSAR models, training a classifier is a matter of changing the underlying model to be used.

In [ ]:
pcm_cls_model = xgboost.XGBClassifier(verbosity=0)

In [37]:
pcm_cls_results, pcm_cls_trained_model = pcm(data=SLC6A4_human_rat,
                                             endpoint='pchembl_value_Mean',
                                             num_points=30,
                                             delta_activity=2,
                                             mol_descriptors='mold2',
                                             mol_descriptor_path=None,
                                             mol_descriptor_chunksize=50000,
                                             prot_sequences_path=None,
                                             prot_descriptors='unirep',
                                             prot_descriptor_path=None,
                                             prot_descriptor_chunksize=50000,
                                             activity_threshold=6.5,
                                             model=pcm_cls_model,
                                             folds=5,
                                             stratify=False,
                                             split_by='Year',
                                             split_year=2013,
                                             test_set_size=0.30,
                                             cluster_method=None,
                                             custom_groups=None,
                                             random_state=1234,
                                             verbose=True)

Loading molecular descriptors: 0it [00:00, ?it/s]

Loading protein descriptors: 0it [00:00, ?it/s]

Fitting model:   0%|          | 0/6 [00:00<?, ?it/s]

In [38]:
pcm_cls_results

,MCC,0:1,ACC,BACC,Sensitivity,Specificity,PPV,NPV,F1,AUC 0,AUC 1
Fold 1,0.593308,528:225,0.837981,0.759495,0.564444,0.954545,0.841060,0.837209,0.675532,0.102689,0.897311
Fold 2,0.600927,545:208,0.847278,0.781528,0.634615,0.928440,0.771930,0.869416,0.696570,0.111133,0.888867
Fold 3,0.554393,530:223,0.824701,0.744297,0.547085,0.941509,0.797386,0.831667,0.648936,0.096438,0.903562
Fold 4,0.582625,539:213,0.840426,0.750965,0.544601,0.957328,0.834532,0.841762,0.659091,0.113164,0.886836
Fold 5,0.608263,538:214,0.848404,0.775857,0.607477,0.944238,0.812500,0.858108,0.695187,0.091148,0.908852
Mean,0.587903,-,0.839758,0.762428,0.579645,0.945212,0.811481,0.847632,0.675063,0.102914,0.897086
SD,0.017147,-,0.007763,0.012999,0.032444,0.009397,0.022986,0.012795,0.017347,0.007671,0.007671
Test set,0.338444,391:241,0.702532,0.645775,0.406639,0.884910,0.685315,0.707566,0.510417,0.272187,0.727813
